In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['yelp-dataset', 'balanced-reviews']


In [2]:
import re
import string
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
reviews_final = pd.read_csv("../input/balanced-reviews/balanced_reviews.csv", index_col=0)

In [4]:
reviews_final.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,fun_bin
17,cHdJXLlKNWixBXpDwEGb_A,1,2015-04-01 16:30:00,7,6BnQwlxRn7ZuWdzninM9sQ,3,I love chinese food and I love mexican food. W...,1,JSrP-dUmLlwZiI7Dp3PQ2A,1
21,Mem13A3C202RzT53npn4NA,9,2017-05-13 10:41:43,6,IPw8yWiyqnfBzzWmypUHgg,5,If you are looking for the best pierogies in P...,9,5JVY32_bmTBfIGpCCsnAfw,1
62,SU56w479vUfFHsvmvQIf7A,6,2016-07-25 03:55:20,5,E4LqIZ7DJd_R4ZHSNKx4RQ,4,So good! They didn't make it to 5 stars due to...,7,DoRCeCcJbrsM2BiAKj3trA,1
126,tjAeaGdxf7I4xN9M7wGJNQ,4,2014-07-13 14:32:56,5,TaoaX7MqCujFRNaJBns2Sw,5,While the prices are a bit high for a make-you...,8,x37OyP--VEFE5p-xreplYA,1
246,FhIeCF6QrsLaRvAeu0oEPQ,4,2013-06-24 06:42:29,5,3Qc49B7dA0ONmCxrn5iwCQ,2,OVERALL: The food isn't good (I explain below)...,13,2k8OVAPxlXHsA5X6EIoQpQ,1


In [5]:
df_reviews = reviews_final[['funny','text', 'fun_bin']]

In [6]:
pd.set_option('display.max_colwidth', -1)
df_reviews.head()

,funny,text,fun_bin
17,7,"I love chinese food and I love mexican food. What can go wrong? A couple of things. First things first, this place is more of a ""rice bowl"" kind of place. I thought it was going to be more diverse as far as the menu goes, but its mainly rice bowls you get with different kinds of meats. The ordering was a little confusing at first, but one of the employees helped us out and I got the 2-item bowl and got the jade chicken and hengrenade chicken with all rice(jerk). I also ordered a jade chicken quesadilla on the side.\n\nI'm gonna admit, this place looks kinda dirty. I don't think Arizona uses those health department letter grade system like California does, but if I were to just judge by how it looked inside, i'd give it a ""C"" grade lol. We waited for about 15 minutes or so and finally got our food. We took it to go and ate at our hotel room. \n\nMmmm... the food was just alright. The jade chicken was nothing special. It tasted like any generic chinese fast food orange chicken/sesame chicken variant. The hengrenade chicken, although was the less spicier version of the jerk chicken, was still pretty spicy for me. Just be warned the jerk chicken is super spicy. If you aren't sure, ask for a sample at the restaurant before ordering, but it was way too spicy for me. \n\nThe jade chicken quesadilla was decent, but nothing special. Just imagine orange chicken in between a tortilla and cheese. A friend of mine ordered a jade chicken burrito and we were confused when we pulled it out of the bag because it was literally the size of Mcdonald's apple pie. If you order the burrito, be warned that it's a burrito for gnomes and smurfs, but he said it was tasty. \n\nThey provide a snicker doodle sugar cookie for each meal and it was decent, again nothing special. \n\nNot gonna lie, the next day my stomach felt like a little mexican dude and chinese dude were wrestling and throwing molotov cocktails inside. I used the bathroom like 5 times. I don't recommend eating this place if you have a lot to do the next day.",1
21,6,"If you are looking for the best pierogies in Pittsburgh, this is your place. There are a few small tables outside but most of the business is carry out. Pierogies Plus wins Best Pierogies every year. Why? Because the owner is from Poland and she is making the real deal pierogies. The best part is that they are hand pinched by a group of older Polish and Hungarian women. \nThe biggest seller is potato and cheese but they sell many flavors. They are like plump pillows of softness. You can buy them buy the dozen. You can get them cold to take home and freeze or warm and ready to eat. The warm ones are served with butter and onions. It's definitely a comfort food. The best part is that they ship internationally. Yes, they are that good.",1
62,5,"So good! They didn't make it to 5 stars due to the prices are a bit high for the amount of food and the location is a bit unsavory. \nThe decor and atmosphere was surprisingly nice, from the outside I expected to be more run down inside. The staff was very nice. We were surprised how empty the dining room was for a Friday evening.\nWe got Vegetable Samosas to start then ordered Chicken Tikka Masala, Lamb Rogan Josh, rice and plain Naan. Our only complaint was the lamb could've been more tender but everything was flavorful and delicious. \nI would definitely go again if given the chance.",1
126,5,"While the prices are a bit high for a make-your-own pizza, the taste makes up for it. I love going to Seventh Street market, sitting Not Just Coffee and having a drink while waiting for delicious fresh made pizza from Pure. I've taken this to go as well as eaten inside the market, and I can say that the pizza doesn't do well reheated. So try to eat it fresh while there if possible.\n\nIf one of their specialty pizzas sounds good to you, go for it, as those are definitely a better deal for the amount of toppings you get for the money. I wanted what I wanted, though, so

### Data pre-processing

#### Goals
- Keep punctuation
- Split by ".", "!" to account for misspeling (like "Hi!I went to...")

In [7]:
!pip install spacymoji

In [8]:
import pandas as pd
import numpy as np
from string import punctuation
import os
import spacy
import string
import re
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from spacymoji import Emoji

In [9]:

re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

#nlp = spacy.load("en")
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

def clean_text(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    #text = text.lower()

    # Clean the text
   # text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=\(\)]", " ", text) # keep punctuatuin, numnbers and letters
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " . ", text) #Add space to the dot
    text = re.sub(r"!", " ! ", text) #Add space to the exclamation sign
    text = re.sub(r":", " :", text) #Add space before : sign
    text = re.sub(r";", " ;", text) #Add space before ; sign
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\r", "", text)
    #text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    #text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    # find emojis
    emoji_list = []
    '''
    for word in text.split():
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    emoji_list'''
    #text = text.split()

    return text

my_tok = spacy.load('en')
emoji = Emoji(my_tok)
my_tok.add_pipe(emoji, first=True)
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(clean_text(x))]

def remove_stop_words(tokens): return [tok for tok in tokens if tok not in spacy_stopwords]

In [10]:
text = "I'm soooo excited!!!!!This is 10000% the best place on earth:))))) 😃..."

In [11]:
text

"I'm soooo excited!!!!!This is 10000% the best place on earth:))))) 😃..."

In [12]:
clean_text(text)

'I m soooo excited ! ! ! ! ! This is 10000% the best place on earth :))))) 😃 . . . '

In [13]:
spacy_tok(clean_text(text))

['I',
 'm',
 'soooo',
 'excited',
 '!',
 '!',
 '!',
 '!',
 '!',
 'This',
 'is',
 '10000',
 '%',
 'the',
 'best',
 'place',
 'on',
 'earth',
 ':',
 ')',
 ')',
 ')',
 ')',
 ')',
 '😃',
 '.',
 '.',
 '.']

In [14]:
text2 = "I also ordered a jade chicken quesadilla on the side.\n\nI'm gonna admit, this place looks kinda dirty. I don't think Arizona uses those health department letter grade system like California does, but if I were to just judge by how it looked inside, i'd give it a 'C' grade lol 😃"

In [15]:
remove_stop_words(spacy_tok(clean_text(text2)))

['I',
 'ordered',
 'jade',
 'chicken',
 'quesadilla',
 '.',
 'I',
 'm',
 'gon',
 'na',
 'admit',
 'place',
 'looks',
 'kinda',
 'dirty',
 '.',
 'I',
 'think',
 'Arizona',
 'uses',
 'health',
 'department',
 'letter',
 'grade',
 'system',
 'like',
 'California',
 'I',
 'judge',
 'looked',
 'inside',
 'C',
 'grade',
 'lol',
 '😃']

### Building a vocabulary

In [16]:
text = [clean_text(sub_br(x)) for x in df_reviews.text]
text

['I love chinese food and I love mexican food . What can go wrong? A couple of things . First things first this place is more of a "rice bowl" kind of place . I thought it was going to be more diverse as far as the menu goes but its mainly rice bowls you get with different kinds of meats . The ordering was a little confusing at first but one of the employees helped us out and I got the 2 - item bowl and got the jade chicken and hengrenade chicken with all rice(jerk) . I also ordered a jade chicken quesadilla on the side . I m gonna admit this place looks kinda dirty . I do not think Arizona uses those health department letter grade system like California does but if I were to just judge by how it looked inside i would give it a "C" grade lol . We waited for about 15 minutes or so and finally got our food . We took it to go and ate at our hotel room . Mmmm . . . the food was just alright . The jade chicken was nothing special . It tasted like any generic chinese fast food orange chicken

In [17]:
df_reviews.text = text

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [18]:
# getting a vocabulary of characters
letters = [list(l) for l in df_reviews.text]
vocab = sorted(list(set(np.concatenate(np.array(letters)))))
vocab[:10]

[' ', '!', '"', '#', '$', '%', '&', '(', ')', '*']

In [19]:
vocab2id = {key:i for i, key in enumerate(vocab)}
vocab2id[" "] # I am going to use 0 to pad sequences

0

In [20]:
vocab[:15]

[' ', '!', '"', '#', '$', '%', '&', '(', ')', '*', '+', '-', '.', '0', '1']

In [21]:
# WHat is the 99% quantile of  length of the sentence?
df_reviews['len_text'] = df_reviews['text'].apply(lambda x: len(list(x)))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
df_reviews['len_text'].quantile(0.99)

4289.0

In [23]:
df_reviews['len_text'].quantile(0.95)

2733.0

In [24]:
len(vocab2id)

1632

In [25]:
def pad_seq(x, seq_len=2800, vocab2id=vocab2id):
    x = list(x)
    x = np.array([vocab2id[k] for k in x])
    z = np.zeros(seq_len, dtype=np.int32)
    n = min(seq_len, x.shape[0])
    z[seq_len - n:] = x[0:n]
    return z

In [26]:
df_reviews.text.iloc[0]

'I love chinese food and I love mexican food . What can go wrong? A couple of things . First things first this place is more of a "rice bowl" kind of place . I thought it was going to be more diverse as far as the menu goes but its mainly rice bowls you get with different kinds of meats . The ordering was a little confusing at first but one of the employees helped us out and I got the 2 - item bowl and got the jade chicken and hengrenade chicken with all rice(jerk) . I also ordered a jade chicken quesadilla on the side . I m gonna admit this place looks kinda dirty . I do not think Arizona uses those health department letter grade system like California does but if I were to just judge by how it looked inside i would give it a "C" grade lol . We waited for about 15 minutes or so and finally got our food . We took it to go and ate at our hotel room . Mmmm . . . the food was just alright . The jade chicken was nothing special . It tasted like any generic chinese fast food orange chicken 

In [27]:
x = pad_seq(df_reviews.text.iloc[0])

In [28]:
x[700:1000]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0, 36,  0, 71, 74, 81, 64,  0, 62, 67, 68, 73, 64, 78, 64,
        0, 65, 74, 74, 63,  0, 60, 73, 63,  0, 36,  0, 71, 74, 81, 64,  0,
       72, 64, 83, 68, 62, 60, 73,  0, 65, 74, 74, 63,  0, 12,  0, 50, 67,
       60, 79,  0, 62, 60, 73,  0, 66, 74,  0, 82, 77, 74, 73, 66, 26,  0,
       28,  0, 62, 74, 80, 75, 71, 64,  0, 74, 65,  0, 79, 67, 68, 73, 66,
       78,  0, 12,  0, 33, 68, 77, 78, 79,  0, 79, 67, 68, 73, 66, 78,  0,
       65, 68, 77, 78, 79,  0, 79, 67, 68, 78,  0, 75, 71, 60, 62, 64,  0,
       68, 78,  0, 72, 74, 77, 64,  0, 74, 65,  0, 60,  0,  2, 77, 68, 62,
       64,  0, 61, 74, 82, 71,  2,  0, 70, 68, 73, 63,  0, 74, 65,  0, 75,
       71, 60, 62, 64,  0

In [29]:
# one hot encoding
def seq2matrix(x, vocab_len=55):
    z = np.zeros((x.shape[0], vocab_len))
    z[np.arange(len(x)), x] = 1
    return z

In [30]:
vocab_len = len(vocab)

In [31]:
seq2matrix(x,vocab_len)

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

### Splitting into train and validation sets

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
 X_train, X_valid, y_train, y_valid = train_test_split(df_reviews['text'], df_reviews['fun_bin'], test_size=0.2, random_state=42)

In [34]:
X_train.reset_index(inplace=True, drop=True)
X_valid.reset_index(inplace=True, drop=True)

In [35]:
y_train.reset_index(inplace=True, drop=True)
y_valid.reset_index(inplace=True, drop=True)

In [36]:
X_train[0]

'Came by for a drink the other day and it was lackluster . The highlight was an abundance of parking along with an air conditioned seating area . . . Service by the staff was confusing at best despite being dressed in all black with their name tags . . . . one of the waitresses was named "Drizzle" ( . . . is that a joke?)The milk tea with pearls was best described as tea flavored water with pearls that were too soft . . . There at least was a nice layer of froth on top of my drink . . . '

### Writing a dataset

In [37]:
class YelpDataset(Dataset):
    def __init__(self, x, y, vocab2id, seq_len=2800, vocab_len=1632):
        self.vocab2id = vocab2id
        self.seq_len = seq_len
        self.vocab_len = vocab_len 
        #self.df = df
        self.x = x.values
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = pad_seq(self.x[idx], self.seq_len, self.vocab2id)
        #x = seq2matrix(x, self.vocab_len)
        return x, self.y[idx]
    

In [38]:
train_ds =  YelpDataset(X_train, y_train, vocab2id)
valid_ds =  YelpDataset(X_valid, y_valid, vocab2id)

#### Check training and validation datasets for empty sentences

In [39]:
# training
#neg=[]
#i=0
#for x,y in train_ds:
#    if len(x) <=0:
#        neg.append(i)
#    i+=1

In [40]:
#neg

In [41]:
# validation
#neg=[]
#i=0
#for x,y in valid_ds:
#    if len(x) <=0:
#        neg.append(i)
#    i+=1

In [42]:
#neg

In [43]:
len(y_valid)

30101

In [44]:
batch_size = 2000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [45]:
np.where(df_reviews.isna())

(array([], dtype=int64), array([], dtype=int64))

In [46]:
train_ds[2]

(array([ 0,  0,  0, ..., 64, 64, 63], dtype=int32), 0)

In [47]:
batch_size = 2000
n = len(valid_ds)
train_dl = DataLoader(train_ds, batch_size=batch_size)
valid_dl = DataLoader(valid_ds, batch_size=n)

In [48]:
class CharEmbRNN(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super(CharEmbRNN, self).__init__()
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.hidden_size = hidden_size
        self.linear_i2h = nn.Linear(emb_size + hidden_size, hidden_size)
        self.linear_h2o = nn.Linear(hidden_size, 1)

    def forward(self, x, hidden):
        x = x.long()
        x = self.emb(x)
        combined = torch.cat((x, hidden), 1)
        hidden = torch.tanh(self.linear_i2h(combined))
        output = self.linear_h2o(hidden)
        return output, hidden

    def initHidden(self, bash_size):
        return torch.zeros(bash_size, self.hidden_size)

In [49]:
def get_optimizer(model, lr = 0.01, wd = 0.00001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

In [50]:
def train(model, optim, train_dl):
    model.train()
    total = 0
    sum_loss = 0
    for x, y in train_dl:
        batch = x.shape[0]
        h = model.initHidden(batch)
        loss = 0
        x = x.float()
        y = y.float()
        
        for t in range(x.shape[1]):
            out, h = model(x[:,t], h)
        
        loss = F.binary_cross_entropy_with_logits(out, y.unsqueeze(1))
        optim.zero_grad()
        loss.backward()
        #loss.backward(retain_graph=True)
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
    return sum_loss/total

In [51]:
def val_metric(model, val_dl):
    model.eval()
    x, y = next(iter(val_dl))
    x = x.float()
    y = y.float()
    N = x.shape[0]
    h = model.initHidden(N)
    for t in range(x.shape[1]):
        out, h = model(x[:,t], h)
    loss = F.cross_entropy(out, y)
    _, pred = torch.max(out, 1)
    acc = pred.eq(y).sum().float()/N
    return loss.item(), acc.item()

In [52]:
def train_loop(model, lr, train_dl, val_dl, epochs=20):
    optim = get_optimizer(model, lr =lr, wd = 0.0)
    for i in range(epochs):
        loss = train(model, optim, train_dl)
        val_loss, val_acc = val_metric(model, val_dl)
        if i%5 == 1: print("train loss %.3f val loss %.3f and val accuracy %.3f" % (loss, val_loss, val_acc))

In [53]:
vocab_size = len(vocab)
emb_size = 30
hidden_size = 80
model = CharEmbRNN(vocab_size, emb_size, hidden_size)

In [ ]:
train_loop(model, 0.01, train_dl, valid_dl, epochs=30)